<a href="https://colab.research.google.com/github/Ninevv/RAG_Project/blob/Zineb/Copie_de_Project_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project : RAG

The goal of this project is to create a simple LLMs based RAG module. Obviously the most complex part is how to correctly link all components. We let you free to create your own database. Anything can be ragged, from research paper, songs, subtitles, books,...

This project is voluntarly sparsely helped, as, as engineers, you will dig into lots of existing method, and will need to pick the best one up. We want you to get familiar with the engineering world.

We will give you some recommendations. You will see lots of issues during this project, some you've already seen during Labs, other that are new. So buckle up, read the docs, and RAG your data.

Obviously there are lots of tutorials of the internet that you could just copy paste to get a baseline.


## **We encourage you to do code versioning using Github.**


**Ideal Project Timeline:**

*   Talking and getting to know the Modules. Discussing about the choice of your LLMs and the environment you'll have. Choosing a first set of data to RAG. Setting up your Github (Optional) (1h)
*   Setting up your first RAG Chain using Langchain or other (2-3h)
*   Understand the limitation of your RAG and find enhancements to set up your 2nd RAG. (2h)
*   Unveiling the unknown document, adapt your RAGs (2h)
*   Deploy it (Optional)
*   Begin your presentation (2h)
*   Presentation (5 min/groups)


We'll evaluate your presentation quality, your RAG system's capability, and your progress throughout the project sessions.

**Students who missed the first session will start with a score of 0 in the progress part**. :-)


Presentation:
- The number of slides you can do is unlimited
- You only have 5 min to present your project. We will stop you at 5 min whether you've finished or not
- Your presentation should include:
  - a presentation of your workflow (Agile Methodology, What's the job of each one...)
  - a presentation of your final pipeline with all enhancements done
  - a proof a work of your RAG on your data, and on the unknown data
  - what limitations you have and how to tackle them. For each too obvious limitations (more GPUs, more RAM..) : -1
  - if you've deployed your RAG, a scannable QR code to live test it.

#  Preliminaries : Some useful downloads

We give you some useful frameworks, that you could use to build your RAG.

In [1]:
!pip install -q pypdf python-dotenv
!pip install transformers
!pip install -q datasets loralib sentencepiece
!pip install -q einops accelerate langchain bitsandbytes
!pip install sentence_transformers
!pip install llama-index
!%pip install --upgrade --quiet  langchain langchain-openai faiss-cpu tiktoken


/bin/bash: line 1: fg: no job control


In [2]:
!pip install chromadb

In [3]:
!pip install langchain_openai

In [4]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

# import dotenv

# dotenv.load_dotenv()

··········


# I - Data Sourcing

Data Sourcing for RAG is really simple. Some questions to guide you:

* What data do we need ?
* How can we correctly parse the data ?
* Does the vector index provides a good representation for the vector database ?
* For example, using FAISS, can you easily retrieve your document ?

To begin, pick a simple story and store it in a Vector Database. You have the choice between multiple VectorStores (ChromaDB, QDrant,...)


#Indexing : Loading data #

#PDF#

In [5]:
!pip install pypdf
!pip install rapidocr-onnxruntime

In [6]:
%pip install -qU langchain-text-splitters
from langchain_community.document_loaders import WebBaseLoader

In [49]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("RAG_Data/Definition of literary movement.pdf", extract_images=True)
pages = loader.load_and_split()

In [51]:
pages[0]

Document(page_content='Definition of literary movement  \n  \nA literary movement is a term that brings together authors whose writing style and vision of \nliterature share many common traits. Sometimes, these authors form veritable schools (such as \nnaturalism) with a common project.  \n  \n   \n  \nHumanism  \n  \nHumanism, a 19th -century term, refers to a European literary and philosophical cultural movement \nof the 15th and 16th centuries, corresponding to the Renaissance.  \n  \nIn France, the main figures of humanism as a literary movement are Rabelais (c. 1494 - 1553), Marot \n(1496 - 1544), Montaigne (1533 - 1592) and the Pléiade poets (see below). The Dutchman Erasmus \n(1467 - 1536), however, is the author who most embodies, as a  symbol, European humanism.  \n  \nHumanism is characterized above all by the status it confers on ancient sources. The great authors of \nGreek and Latin antiquity (e.g. Plato, Aristotle, Cicero, Plutarch, Homer, Virgil, etc.) became models \nto

#Indexing : Split#

In [52]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    add_start_index=True
)
all_splits = text_splitter.split_documents(pages)

In [53]:
all_splits[0]
len(all_splits)

61

In [54]:
all_splits[0].page_content
len(all_splits[0].page_content)

954

In [55]:
all_splits[0].metadata

{'source': 'RAG_Data/Definition of literary movement.pdf',
 'page': 0,
 'start_index': 0}

#Indexing: Store#


In [56]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings(),
                                    persist_directory="./chroma_db")

#Retrieval and Generation#

In [57]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [60]:
query = r" What is a literary movement ?"
retrieved_docs = retriever.invoke(query)

In [61]:
len(retrieved_docs)

6

In [62]:
print(retrieved_docs[0].page_content)

Definition of literary movement  
  
A literary movement is a term that brings together authors whose writing style and vision of 
literature share many common traits. Sometimes, these authors form veritable schools (such as 
naturalism) with a common project.  
  
   
  
Humanism  
  
Humanism, a 19th -century term, refers to a European literary and philosophical cultural movement 
of the 15th and 16th centuries, corresponding to the Renaissance.  
  
In France, the main figures of humanism as a literary movement are Rabelais (c. 1494 - 1553), Marot 
(1496 - 1544), Montaigne (1533 - 1592) and the Pléiade poets (see below). The Dutchman Erasmus 
(1467 - 1536), however, is the author who most embodies, as a  symbol, European humanism.  
  
Humanism is characterized above all by the status it confers on ancient sources. The great authors of 
Greek and Latin antiquity (e.g. Plato, Aristotle, Cicero, Plutarch, Homer, Virgil, etc.) became models


In [63]:
# save to disk
# db2 = Chroma.from_documents(docs, embedding_function, persist_directory="./chroma_db")
vectorstore.persist()
docs = vectorstore.similarity_search(query)

In [64]:
docs

[Document(page_content='Definition of literary movement  \n  \nA literary movement is a term that brings together authors whose writing style and vision of \nliterature share many common traits. Sometimes, these authors form veritable schools (such as \nnaturalism) with a common project.  \n  \n   \n  \nHumanism  \n  \nHumanism, a 19th -century term, refers to a European literary and philosophical cultural movement \nof the 15th and 16th centuries, corresponding to the Renaissance.  \n  \nIn France, the main figures of humanism as a literary movement are Rabelais (c. 1494 - 1553), Marot \n(1496 - 1544), Montaigne (1533 - 1592) and the Pléiade poets (see below). The Dutchman Erasmus \n(1467 - 1536), however, is the author who most embodies, as a  symbol, European humanism.  \n  \nHumanism is characterized above all by the status it confers on ancient sources. The great authors of \nGreek and Latin antiquity (e.g. Plato, Aristotle, Cicero, Plutarch, Homer, Virgil, etc.) became models', m

# II - Module Creation

Should you create a RAG Module, you need or not a LLM. We encourage you to test some LLMs (Mistral, LLama, Falcon, Gemma, ...) However, be aware that you won't have the space to run it on this colab.

We highly recommend to use LangChain, to build your Q&A app.

Some Questions to guide you:
* What model is easily accesible
* Are there any existing code to begin with ?
* What about the prompts ?
* What about the document parsing ?


# III - Model Serving (Optional and for the best)

You can inspire yourself from the first hands-on, if your feeling powerful, you can build something using fastapi and push your deployed RAG into a simple github.io instance. Or just use the gradio deploiement framework.

##